In [ ]:
pip install pandas numpy sqlalchemy pymysql matplotlib seaborn scikit-learn joblib


In [ ]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

In [ ]:
import os
from sqlalchemy import create_engine

DB_USER = "root"
DB_PASSWORD = "99054"
DB_HOST = "localhost"
DB_NAME = "IPL_Analysis"

engine = create_engine(
    f"mysql+mysqlconnector://{DB_USER}:{DB_PASSWORD}@{DB_HOST}/{DB_NAME}"
)

In [ ]:
print("hello ipl")

In [ ]:
conn = engine.connect()
print("✅ MySQL Connected Successfully")
conn.close()

In [ ]:
import pandas as pd

pd.read_sql("SHOW TABLES;", engine)

In [ ]:
pd.read_sql("SELECT COUNT(*) FROM matches;", engine)
pd.read_sql("SELECT COUNT(*) FROM players;", engine)
pd.read_sql("SELECT COUNT(*) FROM deliveries;", engine)

In [ ]:
matches_df = pd.read_csv(
    r"C:\Users\umesh\Downloads\IPL Analysis\matches.csv"
)
players_df = pd.read_csv(
    r"C:\Users\umesh\Downloads\IPL Analysis\players.csv"
)
deliveries_df = pd.read_csv(
    r"C:\Users\umesh\Downloads\IPL Analysis\deliveries.csv"
)

In [ ]:
matches_df = matches_df.drop_duplicates(subset="match_id")
matches_df["winner"] = matches_df["winner"].fillna("No Result")
matches_df["venue"] = matches_df["venue"].fillna("Unknown")

In [ ]:
deliveries_df = deliveries_df.rename(
    columns={
        "over": "over_no",
        "ball": "ball_no"
    }
)

In [ ]:
deliveries_df = deliveries_df[[
    "match_id",
    "inning",
    "over_no",
    "ball_no",
    "batsman_runs",
    "bowler_runs",
    "is_wicket"
]]

deliveries_df.to_sql(
    "deliveries",
    engine,
    if_exists="append",
    index=False
)

In [ ]:
matches_df.to_sql(
    "matches",
    engine,
    if_exists="append",
    index=False
)

In [ ]:
players_df.to_sql(
    "players",
    engine,
    if_exists="append",
    index=False
)

In [ ]:
pd.read_sql("SELECT COUNT(*) FROM matches;", engine)
pd.read_sql("SELECT COUNT(*) FROM players;", engine)
pd.read_sql("SELECT COUNT(*) FROM deliveries;", engine)

In [ ]:
import pandas as pd

query = """
SELECT winner, COUNT(*) AS wins
FROM matches
WHERE winner <> 'No Result'
GROUP BY winner
ORDER BY wins DESC;
"""

df_wins = pd.read_sql(query, engine)
df_wins

In [ ]:
df_wins_sorted = df_wins.sort_values("wins", ascending=False).head(10)

plt.figure(figsize=(10,5))
plt.bar(df_wins_sorted["winner"], df_wins_sorted["wins"])
plt.xticks(rotation=45)
plt.title("Top 10 IPL Teams by Wins")
plt.xlabel("Team")
plt.ylabel("Wins")
plt.show()

In [ ]:
query = """
SELECT season, winner, wins
FROM (
    SELECT season, winner, COUNT(*) AS wins,
           ROW_NUMBER() OVER (PARTITION BY season ORDER BY COUNT(*) DESC) AS rn
    FROM matches
    WHERE winner <> 'No Result'
    GROUP BY season, winner
) t
WHERE rn = 1
ORDER BY season;
"""

top_team_season = pd.read_sql(query, engine)
top_team_season

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10,5))
plt.bar(top_team_season["season"], top_team_season["wins"])
plt.title("Most Dominant Team per Season")
plt.xlabel("Season")
plt.ylabel("Wins")
plt.show()

In [ ]:
query = """
SELECT venue, team, wins
FROM (
    SELECT
        venue,
        winner AS team,
        COUNT(*) AS wins,
        ROW_NUMBER() OVER (PARTITION BY venue ORDER BY COUNT(*) DESC) AS rn
    FROM matches
    WHERE winner <> 'No Result'
    GROUP BY venue, winner
) t
WHERE rn = 1
ORDER BY venue;
"""

top_team_venue = pd.read_sql(query, engine)
top_team_venue

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12,6))
plt.barh(top_team_venue["venue"], top_team_venue["wins"])


for i, row in top_team_venue.iterrows():
    plt.text(
        row["wins"] + 0.2,     
        i,                      
        row["team"],           
        va="center"
    )

plt.xlabel("Wins")
plt.ylabel("Venue")
plt.title("Most Successful Team at Each Venue")
plt.tight_layout()
plt.show()

In [ ]:
query = """
SELECT 
    m.winner AS team,
    SUM(d.batsman_runs) AS total_runs
FROM deliveries d
JOIN matches m
    ON d.match_id = m.match_id
WHERE m.winner <> 'No Result'
GROUP BY m.winner
ORDER BY total_runs DESC;
"""

team_runs_df = pd.read_sql(query, engine)
team_runs_df

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10,5))
plt.bar(team_runs_df["team"], team_runs_df["total_runs"])
plt.xticks(rotation=45)
plt.title("Total Runs Scored by Each Team")
plt.xlabel("Team")
plt.ylabel("Total Runs")
plt.show()

In [ ]:
query = """
SELECT 
    m.winner AS team,
    SUM(d.is_wicket) AS total_wickets
FROM deliveries d
JOIN matches m
    ON d.match_id = m.match_id
WHERE m.winner <> 'No Result'
GROUP BY m.winner
ORDER BY total_wickets DESC;
"""

wickets_df = pd.read_sql(query, engine)
wickets_df

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10,5))
plt.bar(wickets_df["team"], wickets_df["total_wickets"])
plt.xticks(rotation=45)
plt.title("Total Wickets Taken by Each Team")
plt.xlabel("Team")
plt.ylabel("Wickets")
plt.show()

In [ ]:
query = """
SELECT 
    d.match_id,
    SUM(d.batsman_runs) AS total_runs
FROM deliveries d
GROUP BY d.match_id;
"""

runs_per_match = pd.read_sql(query, engine)

avg_runs = runs_per_match["total_runs"].mean()
print("Average runs per match:", avg_runs)

In [ ]:
plt.figure(figsize=(6,4))
plt.hist(runs_per_match["total_runs"], bins=20)
plt.title("Runs Distribution per Match")
plt.xlabel("Runs")
plt.ylabel("Number of Matches")
plt.show()

In [ ]:
query = """
SELECT 
    winner,
    COUNT(*) AS matches
FROM matches
GROUP BY winner;
"""

result_df = pd.read_sql(query, engine)
result_df

In [ ]:
no_result_count = result_df[result_df["winner"] == "No Result"]["matches"].values[0]
print("No Result matches:", no_result_count)

In [ ]:
plt.figure(figsize=(5,4))
plt.bar(["No Result", "Result"],
        [no_result_count, result_df["matches"].sum() - no_result_count])
plt.title("Match Result Distribution")
plt.ylabel("Matches")
plt.show()

In [ ]:
query = """
SELECT
    SUM(CASE WHEN winner = team1 THEN 1 ELSE 0 END) AS home_wins,
    SUM(CASE WHEN winner = team2 THEN 1 ELSE 0 END) AS away_wins
FROM matches
WHERE winner <> 'No Result';
"""

home_away_df = pd.read_sql(query, engine)
home_away_df

In [ ]:
plt.figure(figsize=(5,4))
plt.bar(["Home Wins", "Away Wins"],
        [home_away_df["home_wins"][0], home_away_df["away_wins"][0]])
plt.title("Home vs Away Performance")
plt.ylabel("Wins")
plt.show()

In [ ]:
query = """
SELECT
    match_id,
    SUM(batsman_runs) AS total_runs
FROM deliveries
GROUP BY match_id;
"""

runs_match_df = pd.read_sql(query, engine)
runs_match_df.head()


In [ ]:
plt.figure(figsize=(7,4))
plt.hist(runs_match_df["total_runs"], bins=20)
plt.title("Runs Distribution per Match")
plt.xlabel("Total Runs in Match")
plt.ylabel("Number of Matches")
plt.show()

In [ ]:
query = """
SELECT
    match_id,
    SUM(is_wicket) AS total_wickets
FROM deliveries
GROUP BY match_id;
"""

wickets_match_df = pd.read_sql(query, engine)
wickets_match_df.head()


In [ ]:
plt.figure(figsize=(7,4))
plt.hist(wickets_match_df["total_wickets"], bins=15)
plt.title("Wickets per Match Distribution")
plt.xlabel("Total Wickets in Match")
plt.ylabel("Number of Matches")
plt.show()


In [ ]:
avg_runs = runs_match_df["total_runs"].mean()
print("Average runs per match:", avg_runs)

In [ ]:
high_scoring_matches = runs_match_df[
    runs_match_df["total_runs"] > avg_runs
]

high_scoring_matches.head()

In [ ]:
top10 = high_scoring_matches.sort_values(
    by="total_runs", ascending=False
).head(10)

plt.figure(figsize=(8,4))
plt.bar(top10["match_id"].astype(str), top10["total_runs"])
plt.title("Top 10 High-Scoring Matches")
plt.xlabel("Match ID")
plt.ylabel("Runs")
plt.xticks(rotation=45)
plt.show()

In [ ]:
query = """
SELECT team,
       SUM(wins) AS wins,
       SUM(matches_played) AS matches_played,
       ROUND(SUM(wins) * 100.0 / SUM(matches_played), 2) AS win_percentage
FROM (
    SELECT team1 AS team,
           COUNT(*) AS matches_played,
           SUM(CASE WHEN winner = team1 THEN 1 ELSE 0 END) AS wins
    FROM matches
    GROUP BY team1

    UNION ALL

    SELECT team2 AS team,
           COUNT(*) AS matches_played,
           SUM(CASE WHEN winner = team2 THEN 1 ELSE 0 END) AS wins
    FROM matches
    GROUP BY team2
) t
GROUP BY team
ORDER BY win_percentage DESC
LIMIT 5;
"""

top5_win_df = pd.read_sql(query, engine)
top5_win_df

In [ ]:
plt.figure(figsize=(7,4))
plt.bar(top5_win_df["team"], top5_win_df["win_percentage"])
plt.title("Top 5 Teams by Win Percentage")
plt.xlabel("Team")
plt.ylabel("Win Percentage")
plt.xticks(rotation=30)
plt.show()

In [ ]:
query = """
SELECT
    winner,
    COUNT(*) AS wins
FROM matches
WHERE winner <> 'No Result'
GROUP BY winner
ORDER BY wins DESC;
"""

outcome_df = pd.read_sql(query, engine)
outcome_df

In [ ]:
plt.figure(figsize=(8,4))
plt.bar(outcome_df["winner"], outcome_df["wins"])
plt.title("Match Outcome Distribution by Team")
plt.xlabel("Team")
plt.ylabel("Wins")
plt.xticks(rotation=45)
plt.show()

In [ ]:
query = """
SELECT
    venue,
    COUNT(*) AS total_matches,
    COUNT(DISTINCT winner) AS winning_teams
FROM matches
WHERE winner <> 'No Result'
GROUP BY venue
ORDER BY total_matches DESC;
"""

venue_impact_df = pd.read_sql(query, engine)
venue_impact_df.head()

In [ ]:
plt.figure(figsize=(9,4))
plt.bar(venue_impact_df["venue"], venue_impact_df["winning_teams"])
plt.title("Venue Impact on Match Results")
plt.xlabel("Venue")
plt.ylabel("Number of Different Winning Teams")
plt.xticks(rotation=45)
plt.show()